In [4]:
# from transformers import pipeline
from PIL import Image
"SkalskiP/better-florence-2"


In [44]:
import gradio as gr
import random
import asyncio
import time

with gr.Blocks() as demo:
    with gr.Row():
        
        # msg = gr.Textbox()
        clear = gr.Button("Clear")
        t=gr.Text("")
        image=gr.Image(type='pil')
        chatbot = gr.Chatbot()

        text=[]
        
        def user(user_message, history):
            return "", history + [[user_message, None]]

        async def bot(history):
            bot_message = random.choice(["How are you?", "I love you", "I'm very hungry"])
            # history= ""
            global text
            # print()
            async for character in chain.astream(history):
                text.append([None,character])
                
        def bot2():
            global text
            # print()
            for character in text:
                text.append([None,character])
                time.sleep(0.05)
                yield text
            # async for character in chain.astream(history):
            #     text.append([None,character])


        
                

        image.upload(bot, image, chatbot,queue=False).then(bot2, None, chatbot)
        clear.click(lambda: None, None, chatbot, queue=False)
        # chatbot.change(lambda _ : chatbot.value([["","helllo"]]))

demo.launch()


Running on local URL:  http://127.0.0.1:7899

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/queueing.py", line 580, in process_events
    response = await route_utils.call_process_api(
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/blocks.py", line 1938, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/blocks.py", line 1761, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/components/chatbot.py", line 315, in postprocess
    self._postprocess_chat_messages(message_pair[1]),
  File "/h

In [5]:
from transformers import pipeline
from langchain_core.runnables import RunnableLambda
from langchain_huggingface import HuggingFaceEndpoint
from PIL import Image

pipe1 = pipeline("object-detection", model="facebook/detr-resnet-50")
pipe2 = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
HUGGINGFACEHUB_API_TOKEN='hf_fytqFOBCexHEJOGMFpFHAiUrvnWPaeRU'

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    # max_length=100,
    temperature=0.5,
    streaming=True,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)
def reduce_add(a):
    ll=dict()
    for i in a:
        if i['score'] > 0.89:
            if i['label'] not in ll.keys():
                ll[i['label']] = 1
            else:
                ll[i['label']]+=1
    return "there are \n"+', \n'.join([str(i[1])+' '+i[0]  for i  in ll.items() ])

def image_segmentation_tool(image):
    # image = Image.open(image_path)
    segmentation_results = pipe1(image)
    if reduce_add(segmentation_results) == "there are \n":
        raise Passs()
    return reduce_add(segmentation_results)

def image_caption_tool(image):
    # image = Image.open(image_path)
    segmentation_results = pipe2(image)
    if segmentation_results[0]["generated_text"] == "":
        raise Passs("no result found use different image to create story") 
    return segmentation_results[0]["generated_text"]

from langchain_core.prompts import PromptTemplate


def story_generation_tool(segmentation_results):
    prompt_template = """
    You are a storyteller. Based on the following segmentation results, create a story:
    {segmentation_results}
    Story:
    """
    prompt = PromptTemplate.from_template(prompt_template)
    story = prompt | llm
    return story.invoke(input={"segmentation_results":segmentation_results})

# def translation_tool(english_text):
#     prompt_template = """
#     You are a translator. Translate the following English text to Hindi:
#     {english_text}

#     Translation:
#     """
#     prompt = PromptTemplate.from_template(prompt_template)
#     translation = prompt | llm
#     return translation.invoke(input={"english_text": english_text})


runnable = RunnableLambda(image_segmentation_tool).with_fallbacks([RunnableLambda(image_caption_tool)])
runnable2 = RunnableLambda(story_generation_tool)
# runnable3 = RunnableLambda(translation_tool)

chain = runnable | runnable2

# import gradio as gr

# title = "Image to short Story Generator"
# description = """
# Upload an image, and this app will generate a short story based on the image.
# """


# async def sepia(input_img):
#     chunks=[]
#     async for e in chain.astream(input_img):
#         print('dd')
#         # yield "".join(chunks)
#         chunks.append(e)
#         yield chunks
#         # yield "".join(chunks)

# demo = gr.Interface(sepia, gr.Image(type='pil'), "textarea",title=title,
#     description=description,live=True
# )
# if __name__ == "__main__":
#     demo.launch()

/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (in

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/kar911/.cache/huggingface/token
Login successful


In [34]:
import gradio as gr

with gr.Blocks() as demo:
    with gr.Row():
        chat=gr.ChatInterface()
        image=gr.Image(type='pil')
        clear=gr.ClearButton([chat,image])

        image.upload(sepia,inputs=image,outputs=chat,queue=False)

        def user(user_message, history):
            return "", history + [[user_message, None]]

        def bot(history):
            history[-1][1] = ""
            for character in bot_message:
                history[-1][1] += character
                time.sleep(0.05)
                yield history

        image.submit(user, [chat], [chat], queue=False).then(
            bot, chat, chat
        )
        clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()

TypeError: ChatInterface.__init__() got an unexpected keyword argument 'height'

In [ ]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from threading import Thread

tokenizer = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1")
model = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1", torch_dtype=torch.float16)
model = model.to('cuda:0')

class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [29, 0]
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

def predict(message, history):
    history_transformer_format = history + [[message, ""]]
    stop = StopOnTokens()

    messages = "".join(["".join(["\n<human>:"+item[0], "\n<bot>:"+item[1]])
                for item in history_transformer_format])

    model_inputs = tokenizer([messages], return_tensors="pt").to("cuda")
    streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        model_inputs,
        streamer=streamer,
        max_new_tokens=1024,
        do_sample=True,
        top_p=0.95,
        top_k=1000,
        temperature=1.0,
        num_beams=1,
        stopping_criteria=StoppingCriteriaList([stop])
        )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()

    partial_message = ""
    for new_token in streamer:
        if new_token != '<':
            partial_message += new_token
            yield partial_message

gr.ChatInterface(predict).launch()

In [39]:
class ALLMODELS(Runnable,object):
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = object.__new__(cls)
        return cls._instance
    
    def __init__(self) -> None:
        if not hasattr(self, 'initialized'):  # Ensure models are initialized only once
            self.object_detection = pipeline("object-detection", model="facebook/detr-resnet-50")
            self.initialized = True

    def invoke(self,text):
        return "Invoke method called"+text

In [71]:
# import gradio as gr

# # Initialize a ChatInterface
# def echo(message, history):
#     return message

# demo = gr.ChatInterface(fn=echo, examples=["hello", "hola", "merhaba"], title="Echo Bot")

# # Extract the chat element
# def get_chat_element():
#     return demo.get()

# # Use the chat element as the output for an image input
# image = gr.Image(type="filepath")
# image.upload(get_chat_element, image, demo)

# demo.launch()


def echo(message, history, system_prompt, tokens):
    response = f"System prompt: {system_prompt}\n Message: {message}."
    for i in range(min(len(response), int(tokens))):
        time.sleep(0.05)
        yield response[: i+1]
        
import gradio as gr

# Initialize a Chatbot
with gr.Blocks() as demo:
    with gr.Row():
        image = gr.Image(type="filepath")
        t=gr.Text()
        chatbot = gr.Chatbot()
        

        def get_chat_element(g):
            return chatbot
        
        def new(image,chatbot):
            full=[]
            for i in [("","How are you?"), ("","I love you"), ("","Im very hungry")]:
                print("fdfd")
                time.sleep(1)
                full.append(i)
                chatbot.append(i)
                print(chatbot)
                yield full

        def respond(message, chat_history):
            bot_message = random.choice(["How are you?", "I love you", "I'm very hungry"])
            chat_history.append(bot_message)
            time.sleep(2)
            return  chat_history
        
        def new1(image,chatbot):
            chatbot+=[["",None]]
            return "",chatbot

        def genrrate(chatbot):
            bot_message = random.choice(["How are you?", "I love you", "I'm very hungry"])
            chatbot[-1][1]=""
            for bm in bot_message:
                chatbot[-1][1] +=bm
                yield chatbot

        image.upload(new1, [image,chatbot], chatbot,queue=False).then(genrrate,chatbot,chatbot)

demo.queue()
demo.launch()

Running on local URL:  http://127.0.0.1:7917

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/routes.py", line 737, in predict
    output = await route_utils.call_process_api(
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/blocks.py", line 1938, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/blocks.py", line 1761, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/components/chatbot.py", line 305, in postprocess
    raise TypeError(
TypeError: Expected a list of lists or list of tuples.

In [31]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = random.choice(["How are you?", "I love you", "I'm very hungry"])
        chat_history.append((message, bot_message))
        time.sleep(2)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()


Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.


In [30]:

# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("",object-detection model="facebook/detr-resnet-50")

image = Image.open("football-match.jpg")

# Run the segmentation pipeline
segmentation_results = pipe(image)


# Print segmentation results
for result in segmentation_results:
    if result['score'] > 0.89:
        print(result)



SyntaxError: invalid syntax. Perhaps you forgot a comma? (3018490456.py, line 4)

In [ ]:
def reduce_add(a):
    ll=dict()
    for i in a:
        if i['score'] > 0.89:
            if i['label'] not in ll.keys():
                ll[i['label']] = 1
            else:
                ll[i['label']]+=1
    return "there are \n"+', \n'.join([str(i[1])+' '+i[0]  for i  in ll.items() ])

In [ ]:
s=reduce_add(segmentation_results)

In [ ]:
print(s)

there are 
1 sports ball, 
4 person


In [ ]:
import langchain
langchain.debug = False

In [ ]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint

HUGGINGFACEHUB_API_TOKEN='hf_fytqFOBCexHEJOGMFpFHAiUrvnWPaeRU'
# repo_id ="mistralai/Mistral-7B-Instruct-v0.2"
# repo_id = "HuggingFaceH4/zephyr-7b-beta"
repo_id = "meta-llama/Meta-Llama-3-70B-Instruct"
hf = HuggingFaceEndpoint(repo_id=repo_id, huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN)

llm=ChatHuggingFace(llm=hf)

# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "You are a helpful assistant. Answer all questions to the best of your ability.",
#         ),
#         MessagesPlaceholder(variable_name="messages"),
#     ]
# )

# chain = prompt | llm

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/kar911/.cache/huggingface/token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
class Passs(Exception):
    def __init__(self,message = "The out put is not generrated"):
        self.message = message
        super().__init__(self.message)

In [7]:
HUGGINGFACEHUB_API_TOKEN='hf_fytqFOBCexHEJOGMFpFHAiUrvnWPaeRU'

In [8]:
from transformers import pipeline
from langchain_core.runnables import RunnableLambda
from langchain_huggingface import HuggingFaceEndpoint,HuggingFacePipeline
from PIL import Image

pipe1 = pipeline("object-detection", model="facebook/detr-resnet-50")
pipe2 = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_length=100,
    temperature=0.5,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)
def reduce_add(a):
    ll=dict()
    for i in a:
        if i['score'] > 0.89:
            if i['label'] not in ll.keys():
                ll[i['label']] = 1
            else:
                ll[i['label']]+=1
    return "there are \n"+', \n'.join([str(i[1])+' '+i[0]  for i  in ll.items() ])

def image_segmentation_tool(image: str):
    # image = Image.open(image_path)
    segmentation_results = pipe1(image)
    if reduce_add(segmentation_results) == "there are \n":
        raise Passs()
    return reduce_add(segmentation_results)

def image_caption_tool(image: str):
    # image = Image.open(image_path)
    segmentation_results = pipe2(image)
    if segmentation_results[0]["generated_text"] == "":
        raise Passs("no result found use different image to create story") 
    return segmentation_results[0]["generated_text"]

from langchain_core.prompts import PromptTemplate


def story_generation_tool(segmentation_results):
    prompt_template = """
    You are a storyteller. Based on the following segmentation results, create a story:
    {segmentation_results}

    Story:
    """
    prompt = PromptTemplate.from_template(prompt_template)
    story = prompt | llm
    return story.invoke(input={"segmentation_results":segmentation_results})

def translation_tool(english_text):
    prompt_template = """
    You are a translator. Translate the following English text to Hindi:
    {english_text}

    Translation:
    """
    prompt = PromptTemplate.from_template(prompt_template)
    translation = prompt | llm
    return translation.invoke(input={"english_text": english_text})


runnable = RunnableLambda(image_segmentation_tool).with_fallbacks([RunnableLambda(image_caption_tool)])
runnable2 = RunnableLambda(story_generation_tool)
runnable3 = RunnableLambda(translation_tool)

chain = runnable | runnable2


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
WARNING! max_length is not default parameter.
                    max_length was tra

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/kar911/.cache/huggingface/token
Login successful


In [ ]:
chain.get_graph().print_ascii()

+-------------------------------+  
| image_segmentation_tool_input |  
+-------------------------------+  
                *                  
                *                  
                *                  
        +---------------+          
        | WithFallbacks |          
        +---------------+          
                *                  
                *                  
                *                  
+-------------------------------+  
| Lambda(story_generation_tool) |  
+-------------------------------+  
                *                  
                *                  
                *                  
+------------------------------+   
| story_generation_tool_output |   
+------------------------------+   


In [ ]:
print(runnable3.invoke("""In a bustling world filled with the constant hum of transportation, two trucks and two airplanes ruled the roads and skies. These mighty vehicles were the lifeblood of the economy, carrying goods and people from one place to another. 

    The two trucks, named Bessie and Betty, were old friends. They had been on the road together for years, their engines purring like contented cats as they carried their precious cargo from one destination to another. Bessie was a sturdy flatbed truck, her bed filled with crates and barrels, while Betty was a sleek refrigerated truck, her cold heart keeping perishable goods fresh and delicious. 
 """))

एक प्रसन्नतम जगत में जीवन के साथ-साथ से राहों पर सुनने वाले व्यवसाय के साथ-साथ दो ट्रक्स और दो हाइड्रोपланे राजते थे। ये शक्तिशाली वाहनें आर्थिक जीवन के रहने वाले साथ-साथ गुड्स और लोगों को एक स्थान से दूसरे तक पहुंचाते थे। 

    दोनों ट्रक्स, बेसिए और बेटी, पलती-पलती राहों पर बहुत सालों से साथ रहते थे। अपने महान व्यवसाय को धन्य माये से लिये अपने शराब से बजते थे, जिससे उन्होंने एक स्थान से अन्य स्थान तक अपना महत्वपूर्ण साथ लिया था। बेसिए एक स्थाण्डर्द ट्राक थी, उसका छत्र केसों और बरिलें से भरा था, जैसे 


In [ ]:
def story_generation_tool(segmentation_results):
    prompt_template = """
    You are a storyteller. Based on the following segmentation results, create a story:
    {segmentation_results}

    Story:
    """
    prompt = PromptTemplate.from_template(prompt_template)
    story = prompt | llm
    return story.invoke(input={"segmentation_results":segmentation_results})

In [ ]:
print(chain.invoke("football-match.jpg"))


    In the heart of the city, nestled between towering buildings and bustling streets, was a small, quiet park. The park was a haven for those seeking a moment of peace and tranquility amidst the chaos of urban life. And on this particular sunny afternoon, it was the perfect setting for a friendly game of catch.
    
    Four friends, all wearing bright smiles and relaxed expressions, gathered in the park. They had known each other since childhood, and their bond had only grown stronger with time. Each one carried a unique energy that added to the group's dynamic.
    
    John, the jokester, always had a quip ready to lighten the mood. He tossed the sports ball to Tom, the athlete, who caught it effortlessly with one hand. Tom, in turn, threw the ball back to John, who leapt in the air to catch it. The ball soared through the air, its bright colors glinting in the sunlight.
    
    Sarah, the thoughtful one, watched the game with a contented smile. She was the glue that held the gro

In [ ]:
async for i in chain.astream_events("football-match.jpg",version="v2"):
    print(i)

{'event': 'on_chain_start', 'data': {'input': 'football-match.jpg'}, 'name': 'RunnableSequence', 'tags': [], 'run_id': 'f8d4b471-d5eb-44b7-825a-521db3301bb5', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': 'football-match.jpg'}, 'name': 'RunnableWithFallbacks', 'tags': ['seq:step:1'], 'run_id': '4e3d063f-768c-4cd3-af8a-7ea0a633fa5e', 'metadata': {}, 'parent_ids': ['f8d4b471-d5eb-44b7-825a-521db3301bb5']}
{'event': 'on_chain_start', 'data': {}, 'name': 'image_segmentation_tool', 'tags': [], 'run_id': 'a2ce2e96-5052-4410-bfff-61be5ad552d5', 'metadata': {}, 'parent_ids': ['f8d4b471-d5eb-44b7-825a-521db3301bb5', '4e3d063f-768c-4cd3-af8a-7ea0a633fa5e']}
{'event': 'on_chain_stream', 'run_id': 'a2ce2e96-5052-4410-bfff-61be5ad552d5', 'name': 'image_segmentation_tool', 'tags': [], 'metadata': {}, 'data': {'chunk': 'there are \n1 sports ball, \n4 person'}, 'parent_ids': ['f8d4b471-d5eb-44b7-825a-521db3301bb5', '4e3d063f-768c-4cd3-af8a-7ea0a633fa5e']}
{'event': 'o

In [ ]:
from transformers import TextIteratorStreamer

In [ ]:
import gradio as gr


title = "Image to short Story Generator"
description = """
Upload an image, and this app will generate a short story based on the image.
"""

def sepia(input_img):
    sepia_img=chain.invoke(input_img)
    return sepia_img

demo = gr.Interface(sepia, gr.Image(type='pil'), "textarea",title=title,
    description=description,live=True
)
if __name__ == "__main__":
    demo.launch()

Task exception was never retrieved
future: <Task finished name='Task-50' coro=<Runnable.abatch_as_completed.<locals>.ainvoke() done, defined at /home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/langchain_core/runnables/base.py:786> exception=NameError("name 'pipe5' is not defined")>
Traceback (most recent call last):
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/langchain_core/runnables/base.py", line 797, in ainvoke
    out = await self.ainvoke(input, config, **kwargs)
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/langchain_core/runnables/base.py", line 3996, in ainvoke
    return await self._acall_with_config(
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/langchain_core/runnables/base.py", line 1650, in _acall_with_config
    output = await coro
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/langchain_core/runnables/bas

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
g=pipe5("hello there")

In [ ]:
g["audio"].squeeze()

array([-4.9485516e-06, -3.5621350e-05, -2.8692162e-05, ...,
        4.8048583e-05,  3.1334428e-05, -9.0521671e-06], dtype=float32)

In [ ]:
from scipy.io import wavfile

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# pipe4 = pipeline("text-to-speech", model="facebook/mms-tts-hin")

pipe5 = pipeline("text-to-speech", model="facebook/mms-tts-eng")

/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', '

In [ ]:
async def get_voices():
    voices = await edge_tts.list_voices()
    return {f"{v['ShortName']} - {v['Locale']} ({v['Gender']})": v['ShortName'] for v in voices}

# Text-to-speech function
async def text_to_speech(text, voice, rate, pitch):
    if not text.strip():
        return None, gr.Warning("Please enter text to convert.")
    if not voice:
        return None, gr.Warning("Please select a voice.")
    
    voice_short_name = voice.split(" - ")[0]
    rate_str = f"{rate:+d}%"
    pitch_str = f"{pitch:+d}Hz"
    communicate = edge_tts.Communicate(text, voice_short_name, rate=rate_str, pitch=pitch_str)
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmp_file:
        tmp_path = tmp_file.name
        await communicate.save(tmp_path)
    return tmp_path, None

# Gradio interface function
def tts_interface(text, voice, rate, pitch):
    audio, warning = asyncio.run(text_to_speech(text, voice, rate, pitch))
    return audio, warning

In [ ]:
!pip install soundfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.9/443.9 kB 10.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 9.5 MB/s eta 0:00:00


In [ ]:
import soundfile as sf
# sf.write(
#         filename,
#         audio.samples,
#         samplerate=audio.sample_rate,
#         subtype="PCM_16",
#     )

In [ ]:
import numpy as np
import gradio as gr
# gr.Audio()
def generate_tone(note):
    d=pipe5(note)
    audio = d["audio"].squeeze()
    return (d["sampling_rate"],audio)

demo = gr.Interface(
    generate_tone,
     gr.Textbox(value=1, label="Duration in seconds"),
    gr.Audio(label="Generated Audio",streaming=True,autoplay=True)
            # gr.Markdown(label="Warning", visible=False)
            ,live=True
)
if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7871

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB

To create a public link, set `share=True` in `launch()`.


/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/processing_utils.py:574: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


In [ ]:
print(chain.invoke("test.png"))

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/transforme

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/kar911/.cache/huggingface/token
Login successful

    Once upon a time, in a quaint little village nestled in the heart of a dense forest, there lived a young girl named Lily. She was a kind and gentle soul, always spreading joy and love wherever she went. Her heart was as big as the forest itself, and she had a special friend that embodied her generous spirit.

    This friend was a beautiful stuffed animal, shaped like a bear, with a soft, velvety heart embroidered on its chest. Lily had found the bear in the forest, abandoned and alone, and she had taken it in as her own. She named him Marvin, and they became inseparable.

    Marvin was more than just a toy to Lily. He was her confidant, he

In [ ]:
#use as backup "nlpconnect/vit-gpt2-image-captioning"

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")
image = Image.open("football-match.jpg")
segmentation_results = pipe(image)
segmentation_results[0]["generated_text"]

/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


AttributeError: 'dict' object has no attribute 'generated_text'

In [ ]:
for i in segmentation_results:
    print(i["generated_text"])

a soccer player kicking a soccer ball 


In [ ]:
runnable.invoke("test.png")

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/transforme

'a stuffed animal with a heart on it '

In [ ]:
print(chain.invoke("The-Foolish.jpg"))

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



    Once upon a time, in a land far, far away, there existed four distinct realms, each with its unique characteristics and inhabitants. These realms were known as the Realm of Fire, the Realm of Water, the Realm of Earth, and the Realm of Air.

    The Realm of Fire was a land of passion and energy. Its inhabitants were fiery beings, with red and orange skin, and eyes that shone like molten gold. They lived in volcanic mountains and fiery forests, surrounded by lava rivers. Their days were filled with excitement and adventure, as they explored the depths of their fiery world.

    The Realm of Water was a tranquil and serene place. Its inhabitants were water creatures, with shimmering blue and green scales, and gills that allowed them to breathe underwater. They lived in vast oceans and crystal-clear lakes, surrounded by coral reefs and underwater gardens. Their days were filled with peace and harmony, as they swam gracefully through their watery world.

    The Realm of Earth was a 

In [ ]:
print(chain.invoke("plan.jpg"))

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



    In a bustling world filled with the constant hum of transportation, two trucks and two airplanes ruled the roads and skies. These mighty vehicles were the lifeblood of the economy, carrying goods and people from one place to another.

    The two trucks, named Bessie and Betty, were old friends. They had been on the road together for years, their engines purring like contented cats as they carried their precious cargo from one destination to another. Bessie was a sturdy flatbed truck, her bed filled with crates and barrels, while Betty was a sleek refrigerated truck, her cold heart keeping perishable goods fresh and delicious.

    The two airplanes, named Falcon and Falconess, were equally impressive. They soared through the skies, their wings cutting through the clouds like knives. Falcon was a cargo plane, his hold filled with pallets and containers, while Falconess was a passenger plane, her cabin filled with excited travelers, eager to explore new places.

    Every day, Bess

In [ ]:
HuggingFaceEndpoint()

In [ ]:
"facebook/mms-tts-eng"
"facebook/mms-tts-hin"
"swarpatil9825/opus-mt-en-hi"
"Helsinki-NLP/opus-mt-en-hi"
"barghavani/English_to_Hindi"

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter,TextSplitter
c=RecursiveCharacterTextSplitter(separators=["."],chunk_size= 20,    chunk_overlap= 0)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe3 = pipeline("translation", model="swarpatil9825/opus-mt-en-hi")

/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
ff=pipe3("In a bustling world filled with the constant hum of transportation, two trucks and two airplanes ruled the roads and skies. These mighty vehicles were the lifeblood of the economy, carrying goods and people from one place to another.")

/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
print(ff)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter,TextSplitter
c=RecursiveCharacterTextSplitter(separators=["."],chunk_size= 20,    chunk_overlap= 0)
g=c.split_text("""
In a bustling world filled with the constant hum of transportation, two trucks and two airplanes ruled the roads and skies. These mighty vehicles were the lifeblood of the economy, carrying goods and people from one place to another.

    The two trucks, named Bessie and Betty, were old friends. They had been on the road together for years, their engines purring like contented cats as they carried their precious cargo from one destination to another. Bessie was a sturdy flatbed truck, her bed filled with crates and barrels, while Betty was a sleek refrigerated truck, her cold heart keeping perishable goods fresh and delicious.

    The two airplanes, named Falcon and Falconess, were equally impressive. They soared through the skies, their wings cutting through the clouds like knives. Falcon was a cargo plane, his hold filled with pallets and containers, while Falconess was a passenger plane, her cabin filled with excited travelers, eager to explore new places.

    Every day, Bessie and Betty would make their way along the highways, their engines rumbling as they climbed hills and descended into valleys. They would pass each other on the road, exchanging friendly honks and waves, before continuing on their separate journeys.

    Meanwhile, Falcon and Falconess would take to the skies, their powerful engines roaring as they ascended into the heavens. They would fly in formation, their wings glinting in the sun as they cruised above the world, before parting ways to deliver their cargo to their respective destinations.

    And so, the two trucks and two airplanes continued their daily routine, their engines never faltering, their spirits never dimming. They were the unsung heroes of the transportation world, the unseen workers keeping the economy running smoothly. And they were proud to do it, day in and day out.
""")

In [ ]:
tet=[]

In [ ]:
async for i in runnable4.abatch_as_completed(inputs=[*g]):
    tet.append(i)

In [ ]:
tet.sort(key=lambda x:x[0])

In [ ]:

from langchain_huggingface import HuggingFaceEndpoint
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
HUGGINGFACEHUB_API_TOKEN='hf_fytqFOBCexHEJOGMFpFHAiUrvnWPaeRU'
llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.5,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
    streaming=True
)

prompt_template = """
You are a storyteller. Based on the following segmentation results, create a story:
{segmentation_results}

Story:
"""

from langchain_core.prompts import PromptTemplate

segmentation_results="""there are 
1 football, 
4 person"""

prompt = PromptTemplate.from_template(prompt_template)
story = prompt | llm
chunks = []
async for e in story.astream(input={"segmentation_results":segmentation_results}):
    chunks.append(e)
    print(e,end="",flush=True)
# llm.astream("")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/kar911/.cache/huggingface/token
Login successful
In the heart of the bustling city, there was a small, quiet park where people went to escape the noise and chaos of urban life. The park was surrounded by towering buildings, but in its center, there was a lush green field where children played and adults relaxed.

One sunny afternoon, four friends, John, Mike, Tom, and Dave, gathered around the park's only football goalpost. They had grown up together, playing football in this very park since they were young boys. Now, as men, they still found joy in the simple game that had brought them together for so many years.

John, the tallest and strongest of the group, was the goalkeeper. Mike, the fast

In [4]:
chain.invoke("")

NameError: name 'chain' is not defined

In [ ]:
import gradio as gr
import asyncio

title = "Image to short Story Generator"
description = """
Upload an image, and this app will generate a short story based on the image.
"""

# v=0



import asyncio
Full=[]
# Declare chunks as global
chunks = []

async def sepia(input_img):
    global chunks
    v = 0
    Full = []
    async for e in story.astream(input={"segmentation_results": input_img}):
        if v // 60 == 0:
            "".join(chunks)
            chunks.clear()
        chunks.append(e)
        Full.extend(chunks)
        v += 1
        yield "".join(Full)

async def sepia1(input_img):
    global chunks
    v = 0
    Full = []
    async for e in story.astream(input={"segmentation_results": input_img}):
        if v // 30 == 0:
            "".join(chunks)
            chunks.clear()
        chunks.append(e)
        Full.extend(chunks)
        v += 1
        yield "".join(Full)

async def combined_sepia_functions(input_img):
    sepia_gen = sepia(input_img)
    sepia1_gen = sepia1(input_img)
    
    while True:
        try:
            result1 = await sepia_gen.__anext__()
        except StopAsyncIteration:
            result1 = None

        try:
            result2 = await sepia1_gen.__anext__()
        except StopAsyncIteration:
            result2 = None

        if result1 is None and result2 is None:
            break

        yield (result1, result2)

# Example usage:
# async for text, audio in combined_sepia_functions(input_img):
#     print("Text:", text)
#     print("Audio:", audio)


# Example usage:
# async for text, audio in combined_sepia_functions(input_img):
#     print("Text:", text)
#     print("Audio:", audio)

async def sepia(input_img):
    v=0
    async for e in chain.astream(input={"segmentation_results":input_img}):
        if v//60== 0:
            "".join(chunks)
            chunks.clear()
        chunks.append(e)
        Full.extend(chunks)
        v=+1
        yield "".join(Full)
        # yield e

demo = gr.Interface(sepia, gr.Image(type='pil'), "textarea",title=title,
    description=description,live=True
)
if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/queueing.py", line 541, in process_events
    response = await route_utils.call_process_api(
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/blocks.py", line 1928, in process_api
    result = await self.call_function(
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/blocks.py", line 1526, in call_function
    prediction = await utils.async_iteration(iterator)
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/utils.py", line 657, in async_iteration
    return await iterator.__anext__()
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# pipe4 = pipeline("text-to-speech", model="facebook/mms-tts-hin")

pipe5 = pipeline("text-to-speech", model="facebook/mms-tts-eng")

Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

In [ ]:
from langchain_core.runnables import RunnableLambda

def generate_tone1(note):
    d= pipe5(note)
    return (d["sampling_rate"],d["audio"].squeeze())


async def generate_tone(note):
    d=  pipe5(note)
    return (d["sampling_rate"],d["audio"].squeeze())
runabl=RunnableLambda(func=generate_tone1,afunc=generate_tone)
runabl.astream("Default implementation of astream, which calls ainvoke. Subclasses should override this method if they support streaming output.")
import numpy as np
import gradio as gr
# gr.Audio()
# def generate_tone(note):
#     d=pipe5(note)
#     audio = d["audio"].squeeze()
#     return (d["sampling_rate"],audio)

from langchain_text_splitters import RecursiveCharacterTextSplitter,TextSplitter

# g=c.split_text("""
# In a bustling world filled with the constant hum of transportation, two trucks and two airplanes ruled the roads and skies. These mighty vehicles were the lifeblood of the economy, carrying goods and people from one place to another.

#     The two trucks, named Bessie and Betty, were old friends. They had been on the road together for years, their engines purring like contented cats as they carried their precious cargo from one destination to another. Bessie was a sturdy flatbed truck, her bed filled with crates and barrels, while Betty was a sleek refrigerated truck, her cold heart keeping perishable goods fresh and delicious.

#     The two airplanes, named Falcon and Falconess, were equally impressive. They soared through the skies, their wings cutting through the clouds like knives. Falcon was a cargo plane, his hold filled with pallets and containers, while Falconess was a passenger plane, her cabin filled with excited travelers, eager to explore new places.

#     Every day, Bessie and Betty would make their way along the highways, their engines rumbling as they climbed hills and descended into valleys. They would pass each other on the road, exchanging friendly honks and waves, before continuing on their separate journeys.

#     Meanwhile, Falcon and Falconess would take to the skies, their powerful engines roaring as they ascended into the heavens. They would fly in formation, their wings glinting in the sun as they cruised above the world, before parting ways to deliver their cargo to their respective destinations.

#     And so, the two trucks and two airplanes continued their daily routine, their engines never faltering, their spirits never dimming. They were the unsung heroes of the transportation world, the unseen workers keeping the economy running smoothly. And they were proud to do it, day in and day out.
# """)

async def generate_tone1(text):
    c=RecursiveCharacterTextSplitter(separators=["."],chunk_size= 20,    chunk_overlap= 0)
    async for i in runabl.astream(text):
        return i


demo = gr.Interface(
    generate_tone1,
     gr.Textbox( label="Duration in seconds"),
    gr.Audio(label="Generated Audio",streaming=True,autoplay=True)
            # gr.Markdown(label="Warning", visible=False)
            ,live=True
)
if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/processing_utils.py:574: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


In [3]:
from langchain_core.runnables import RunnableLambda
def generate_tone1(note):
    d = pipe5(note)
    return (d["sampling_rate"], d["audio"].squeeze())

async def generate_tone(note):
    d = pipe5(note)
    return (d["sampling_rate"], d["audio"].squeeze())

runabl = RunnableLambda(func=generate_tone1, afunc=generate_tone)

In [ ]:
runabl.astream()
runabl.abatch_as_completed()

In [ ]:
from langchain_core.runnables import RunnableLambda
import numpy as np
import gradio as gr
from langchain_text_splitters import RecursiveCharacterTextSplitter, TextSplitter

async def generate_tone2(note):
    d = pipe5(note)
    return (d["sampling_rate"], d["audio"].squeeze())

def generate_tone1(note):
    d = pipe5(note)
    return (d["sampling_rate"], d["audio"].squeeze())

async def generate_tone(note):
    return await generate_tone2(note)

runabl = RunnableLambda(func=generate_tone1, afunc=generate_tone)

async def generate_tone1(text):
    c = RecursiveCharacterTextSplitter(separators=["."], chunk_size=20, chunk_overlap=0)
    chunks = c.split_text(text=text)
    async for i in runabl.abatch_as_completed(chunks):
        yield (16000,i)

demo = gr.Interface(
    fn=generate_tone1,
    inputs=gr.Textbox(label="Duration in seconds"),
    outputs=gr.Audio(label="Generated Audio", streaming=True, autoplay=True),
    live=True
)

demo.output_components.end

if __name__ == "__main__":
    demo.launch()
    # kÄrtiːk


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [4]:
async def generate_tone2(note):
    d = pipe5(note)
    return (d["sampling_rate"], d["audio"].squeeze())

def generate_tone1(note):
    d = pipe5(note)
    return (d["sampling_rate"], d["audio"].squeeze())

async def generate_tone(note):
    yield generate_tone2(note)

runabl = RunnableLambda(func=generate_tone1, afunc=generate_tone)

In [5]:
text="""
In a bustling world filled with the constant hum of transportation, two trucks and two airplanes ruled the roads and skies. These mighty vehicles were the lifeblood of the economy, carrying goods and people from one place to another.

    The two trucks, named Bessie and Betty, were old friends. They had been on the road together for years, their engines purring like contented cats as they carried their precious cargo from one destination to another. Bessie was a sturdy flatbed truck, her bed filled with crates and barrels, while Betty was a sleek refrigerated truck, her cold heart keeping perishable goods fresh and delicious.
"""

In [ ]:
c = RecursiveCharacterTextSplitter(separators=["."], chunk_size=20, chunk_overlap=0)
chunks = c.split_text(text=text)
for i in runabl.abatch_as_completed(chunks):
    print(i)

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [7]:
c=RecursiveCharacterTextSplitter(separators=["."],chunk_size= 20,    chunk_overlap= 0)

NameError: name 'RecursiveCharacterTextSplitter' is not defined

In [6]:
g=c.split_text("""
In a bustling world filled with the constant hum of transportation, two trucks and two airplanes ruled the roads and skies. These mighty vehicles were the lifeblood of the economy, carrying goods and people from one place to another.

    The two trucks, named Bessie and Betty, were old friends. They had been on the road together for years, their engines purring like contented cats as they carried their precious cargo from one destination to another. Bessie was a sturdy flatbed truck, her bed filled with crates and barrels, while Betty was a sleek refrigerated truck, her cold heart keeping perishable goods fresh and delicious.

    The two airplanes, named Falcon and Falconess, were equally impressive. They soared through the skies, their wings cutting through the clouds like knives. Falcon was a cargo plane, his hold filled with pallets and containers, while Falconess was a passenger plane, her cabin filled with excited travelers, eager to explore new places.

    Every day, Bessie and Betty would make their way along the highways, their engines rumbling as they climbed hills and descended into valleys. They would pass each other on the road, exchanging friendly honks and waves, before continuing on their separate journeys.

    Meanwhile, Falcon and Falconess would take to the skies, their powerful engines roaring as they ascended into the heavens. They would fly in formation, their wings glinting in the sun as they cruised above the world, before parting ways to deliver their cargo to their respective destinations.

    And so, the two trucks and two airplanes continued their daily routine, their engines never faltering, their spirits never dimming. They were the unsung heroes of the transportation world, the unseen workers keeping the economy running smoothly. And they were proud to do it, day in and day out.
""")
g

NameError: name 'c' is not defined

In [ ]:
joinchunks

['Once',
 ' upon',
 ' a',
 ' time',
 ',',
 ' in',
 ' a',
 ' small',
 ' village',
 ' nest',
 'led',
 ' between',
 ' rolling',
 ' hills',
 ' and',
 ' a',
 ' spark',
 'ling',
 ' river',
 ',',
 ' there',
 ' was',
 ' a',
 ' group',
 ' of',
 ' four',
 ' friends',
 ' named',
 ' Tom',
 ',',
 ' Jerry',
 ',',
 ' Mike',
 ',',
 ' and',
 ' Sam',
 '.',
 ' They',
 ' were',
 ' in',
 'separ',
 'able',
 ' since',
 ' childhood',
 ' and',
 ' shared',
 ' a',
 ' common',
 ' passion',
 ' for',
 ' sports',
 ',',
 ' especially',
 ' football',
 '.',
 ' Every',
 ' evening',
 ',',
 ' after',
 ' their',
 ' cho',
 'res',
 ' were',
 ' done',
 ',',
 ' they',
 ' would',
 ' gather',
 ' in',
 ' the',
 ' open',
 ' field',
 ' near',
 ' the',
 ' village',
 ' square',
 ' to',
 ' play',
 ' a',
 ' game',
 ' of',
 ' football',
 '.',
 '\n',
 '\n',
 'Tom',
 ',',
 ' the',
 ' tall',
 'est',
 ' and',
 ' strongest',
 ' of',
 ' the',
 ' group',
 ',',
 ' was',
 ' the',
 ' team',
 "'",
 's',
 ' center',
 ' forward',
 '.',
 ' Jerry',
 '

In [ ]:
import asyncio


async def async_generator():
    for i in range(3):
        await asyncio.sleep(1)
        yield i*i


async def main():
    async for i in async_generator():
        print(i)


In [ ]:
loop1 = asyncio.get_event_loop()
try:
    loop1.run_until_complete(main())
finally:
    loop1.run_until_complete(loop.shutdown_asyncgens())  # see: https://docs.python.org/3/library/asyncio-eventloop.html#asyncio.loop.shutdown_asyncgens
    loop1.close()

RuntimeError: This event loop is already running

In [15]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# pipe4 = pipeline("text-to-speech", model="facebook/mms-tts-hin")

pipe5 = pipeline("text-to-speech", model="facebook/mms-tts-eng")

Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

In [4]:
import gradio as gr

def generate_tone1(note):
    d = pipe5(note)
    return (d["sampling_rate"], d["audio"].squeeze())

with gr.Blocks() as demo:
    with gr.Row():
        duration_input = gr.Textbox(label="Duration in seconds")
        generated_audio = gr.Audio(label="Generated Audio", autoplay=True)
    
    # duration_input.change(generate_tone1, inputs=duration_input, outputs=generated_audio)
    @gr.on(triggers=generated_audio.stop, outputs=generated_audio)
    def greet():
        return pq.get()
        # yield pipe5("")

demo.launch()

TypeError: 'method' object is not iterable

In [4]:
from langchain_core.runnables import RunnableLambda

In [2]:
async def generate_tone2(note):
    d = pipe5(note)
    return (d["sampling_rate"], d["audio"].squeeze())

def generate_tone1(note):
    d = pipe5(note)
    return (d["sampling_rate"], d["audio"].squeeze())

async def generate_tone(note):
    yield await generate_tone2(note)

runabl = RunnableLambda(func=generate_tone1,afunc=generate_tone)
# runabl.batch_as_completed()

NameError: name 'RunnableLambda' is not defined

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from queue import Queue

batch_size = 3
pq = Queue()

def process_batch(batch):
    results = []
    for j in batch:
        j = (j[0]+i, j[1])
        results.append(j)
    return results

with ThreadPoolExecutor() as executor:
    futures = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i+batch_size]
        futures.append(executor.submit(process_batch, batch))

    for future in concurrent.futures.as_completed(futures):
        for result in future.result():
            pq.put_nowait(result)

In [152]:

batch_size = 3
for i in range(0, len(chunks), batch_size):
    batch = chunks[i:i+batch_size]
    for j in runabl.batch_as_completed(batch):
        # print(j)
        j=(j[0]+i,j[1])
        pq.put_nowait(j)
        # print(j)
        # print(i+batch_size,batch)

# remaining = len(chunks) % batch_size
# if remaining > 0:
#     batch = chunks[-remaining:]
#     print(f"Processing remaining elements: {batch}")
#     for j in runabl.batch_as_completed(batch):
#         # print(j)
#         j=(j[0]+i,j[1])
#         pq.put_nowait(j)
    # Do something with the remaining elements
    # print(i)

In [28]:

import gradio as gr
import time
from langchain_text_splitters import RecursiveCharacterTextSplitter, TextSplitter
c = RecursiveCharacterTextSplitter(separators=["."], chunk_size=20, chunk_overlap=0)
import queue
from asyncio.queues import PriorityQueue
import asyncio

pq = PriorityQueue(30)
chunks = c.split_text(text=text)
chunks.remove(".")

In [14]:
chunks

['\n\nIn a bustling world filled with the constant hum of transportation, two trucks and two airplanes ruled the roads and skies',
 '. These mighty vehicles were the lifeblood of the economy, carrying goods and people from one place to another',
 '.\n\n    The two trucks, named Bessie and Betty, were old friends',
 '. They had been on the road together for years, their engines purring like contented cats as they carried their precious cargo from one destination to another',
 '. Bessie was a sturdy flatbed truck, her bed filled with crates and barrels, while Betty was a sleek refrigerated truck, her cold heart keeping perishable goods fresh and delicious',
 '.\n\n    The two airplanes, named Falcon and Falconess, were equally impressive',
 '. They soared through the skies, their wings cutting through the clouds like knives',
 '. Falcon was a cargo plane, his hold filled with pallets and containers, while Falconess was a passenger plane, her cabin filled with excited travelers, eager to 

In [94]:
pq.get_nowait()

QueueEmpty: 

In [130]:
while not pq.empty():
    print(pq.get_nowait(),"--")
    if pq.empty :
        break

In [1]:

import gradio as gr
import time
from langchain_text_splitters import RecursiveCharacterTextSplitter, TextSplitter
c = RecursiveCharacterTextSplitter(separators=["."], chunk_size=20, chunk_overlap=0)
import queue
from asyncio.queues import PriorityQueue
import asyncio

pq = PriorityQueue(30)
# chunks = c.split_text(text=text)
# chunks.remove(".")
# def generate_tone1(note):
#     d = pipe5(note)
#     return (d["sampling_rate"], d["audio"].squeeze())

with gr.Blocks() as demo:
    with gr.Row():
        story = gr.TextArea(label="Duration in seconds",show_copy_button=True)
        image=gr.Image(type='pil')
        narrate = gr.Audio(label="Generated Audio", autoplay=True,streaming=True)
    
    # async def greet1():
    #     d= await runabl.ainvoke(chunks[2])
    #     print(type(d))
    #     return  d

        def sepia(input_img):
            # chunks=[]
            # async for e in chain.astream(input_img):
            #     chunks.append(e)
            #     yield "".join(chunks)
            #             # chunks=[]
            # for e in chain.astream(input_img):
            #     chunks.append(e)
            #     yield "".join(chunks)

            return chain.ainvoke(input_img)
        # chian

        def get2(text):
            batch_size = 3
            # pq = PriorityQueue(30)
            chunks = c.split_text(text=text)
            chunks.remove(".")
            for i in range(0, len(chunks), batch_size):
                batch = chunks[i:i+batch_size]
                # iterating the output from batch completion of langcahin.Runnable = runabl
                for j in runabl.batch_as_completed(batch):
                    # print(j)
                    j=(j[0]+i,j[1])
                    # appending the obejct in order
                    pq.put_nowait(j)
                for k in range(3):
                    yield pq.get_nowait()[1]

# get async function for precessing the text in batchhes of 3
        gr.on(fn=sepia,triggers=image.upload,inputs=image,outputs=story).then(fn=get2,inputs=story ,outputs=narrate).then(lambda _ : pq.clear())

    # gr.on()


        async def get():
            global chunks,runabl
            batch_size = 3
            for i in range(0, len(chunks), batch_size):
                batch = chunks[i:i+batch_size]
                # iterating the output from batch completion of langcahin.Runnable = runabl
                for j in runabl.batch_as_completed(batch):
                    # print(j)
                    j=(j[0]+i,j[1])
                    # appending the obejct in order
                    pq.put_nowait(j)
                    print("hello")
                yield
            print("hello")
        
        
        async def get1():
            get()
    # def on_select(value, evt: gr.EventData):
    #     return f"The {evt.target} component was selected, and its value was {value}."
    
    # def bet():
    #     i=0
    #     if i == 0 :
    #         time.sleep(4)
    #     else:
    #         return pq.get_nowait()[1]


    # Do something with the batch, e.g. insert_posts(batch)

# Handle any remaining elements
    # remaining = len(records) % batch_size
    # if remaining > 0:
    #     batch = records[-remaining:]
    #     print(f"Processing remaining elements: {batch}")
        # Do something with the remaining elements

# runabl.batch_as_completed

    # gr.EventData
    # gr.on(triggers=duration_input.change,)
        # def next():
        #     print("hello1")
        #     try:
        #         f=pq.get_nowait()[1]
        #     except asyncio.queues.QueueEmpty:
        #         return None
        #         # generated_audio.pause(lambda _ : 1)
        #     return f

    # duration_input.attach_load_event()
    # duration_input.change(fn=get).then(fn=lambda _ : time.sleep(4),queue=False).fn() .then(fn=lambda _ : pq.get_nowait()[1],outputs=generated_audio)
    # duration_input.change(fn=get).then(fn=next,outputs=generated_audio)
    # generated_audio.(fn=lambda _ : pq.get_nowait(),outputs=generated_audio)
    # generated_audio.stop(fn=next,outputs=generated_audio)

    # duration_input.change(generate_tone1, inputs=duration_input, outputs=generated_audio)
    # @gr.on(triggers=duration_input.change, outputs=generated_audio)

        # yield pipe5("")

demo.launch()

/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: 'method' object is not iterable

In [24]:
from transformers import pipeline
from langchain_core.runnables import RunnableLambda
from langchain_huggingface import HuggingFaceEndpoint
from PIL import Image

pipe1 = pipeline("object-detection", model="facebook/detr-resnet-50")
pipe2 = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"


llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.5,
    streaming=True
)
def reduce_add(a):
    ll=dict()
    for i in a:
        if i['score'] > 0.89:
            if i['label'] not in ll.keys():
                ll[i['label']] = 1
            else:
                ll[i['label']]+=1
    return "there are \n"+', \n'.join([str(i[1])+' '+i[0]  for i  in ll.items() ])

def image_segmentation_tool(image: str):
    # image = Image.open(image_path)
    segmentation_results = pipe1(image)
    if reduce_add(segmentation_results) == "there are \n":
        raise Passs()
    return reduce_add(segmentation_results)

def image_caption_tool(image: str):
    # image = Image.open(image_path)
    segmentation_results = pipe2(image)
    if segmentation_results[0]["generated_text"] == "":
        raise Passs("no result found use different image to create story") 
    return segmentation_results[0]["generated_text"]

from langchain_core.prompts import PromptTemplate


def story_generation_tool(segmentation_results):
    prompt_template = """
    You are a storyteller. Based on the following segmentation results, create a story:
    {segmentation_results}
    Story:
    """
    prompt = PromptTemplate.from_template(prompt_template)
    story = prompt | llm
    return story.invoke(input={"segmentation_results":segmentation_results})

# def translation_tool(english_text):
#     prompt_template = """
#     You are a translator. Translate the following English text to Hindi:
#     {english_text}

#     Translation:
#     """
#     prompt = PromptTemplate.from_template(prompt_template)
#     translation = prompt | llm
#     return translation.invoke(input={"english_text": english_text})


runnable = RunnableLambda(image_segmentation_tool).with_fallbacks([RunnableLambda(image_caption_tool)])
runnable2 = RunnableLambda(story_generation_tool)
# runnable3 = RunnableLambda(translation_tool)

chain = runnable | runnable2

import gradio as gr

title = "Image to short Story Generator"
description = """
Upload an image, and this app will generate a short story based on the image.
"""


def sepia(input_img):
    chunks=[]
    for e in chain.astream(input_img):
        chunks.append(e)
        yield "".join(chunks)

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
async for i in sepia("football-match.jpg"):
    print(i)


Four friends, John, Mike, Sarah, and Tom, gathered in the park for their weekly pick-up soccer game. The sun was setting, casting long shadows over the grassy field. John, the team's goalkeeper, stood guard at the goalposts, while Mike and Tom took positions as midfielders. Sarah, the team's star striker, waited eagerly on the sidelines, ready to make her moves.

As they kicked off, the game began in earnest. Mike and Tom passed the ball back and forth, weaving in and out of their opponents' defense. Sarah made a brilliant run towards the goal, but was tripped up just as she was about to shoot. The referee blew his whistle, awarding a free kick to their team.

John stepped up, the ball at his feet. The opposing team's goalkeeper advanced, ready for the challenge. John took a deep breath, focusing on his aim. With a powerful kick, the ball soared through the air, arcing towards the goal. The goalkeeper leapt, but was unable to reach it in time. The ball bounced off the crossbar and int

In [28]:
with gr.Blocks() as demo:
    with gr.Row():
        story = gr.TextArea(label="Duration in seconds",show_copy_button=True)
        image=gr.Image(type='pil')
        narrate = gr.Audio(label="Generated Audio", autoplay=True,streaming=True)
        print(type(image))
        # gr.on(fn=sepia,triggers=image.upload,inputs=image,outputs=story)
        # print(sepia(image))

demo.launch()

<class 'gradio.components.image.Image'>
Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [11]:

        gr.on(fn=sepia,triggers=image.upload,inputs=image,outputs=story)

TypeError: 'method' object is not iterable

In [10]:
async for e in chain.astream("football-match.jpg"):
        # chunks.append(e)
        print(e)


Once upon a time in a small town nestled between rolling hills and a sparkling lake, there was a beautiful park where the community gathered to play and enjoy the sunshine. Four friends, Jack, Lily, Tom, and Sam, were some of the most dedicated park-goers. They spent their afternoons playing various games and sports, always encouraging each other to try new things and improve their skills.

One sunny afternoon, as they were playing catch with a sports ball, Jack suggested they try something new. "Why don't we start a friendly soccer match?" he proposed. The others agreed, and they quickly set up goals using nearby trees and started the game.

Jack, being the most experienced soccer player, took the position of goalkeeper. Lily, with her quick reflexes, became a formidable defender. Tom, known for his agility and speed, took the role of a midfielder. And Sam, the newest member of their group, was given the position of a forward.

The game began, and despite Sam's initial struggles, the

In [8]:
import gradio as gr
import time
from langchain_text_splitters import RecursiveCharacterTextSplitter, TextSplitter
c = RecursiveCharacterTextSplitter(separators=["."], chunk_size=20, chunk_overlap=0)
import queue
from asyncio.queues import PriorityQueue
import asyncio

pq = PriorityQueue(30)

with gr.Blocks() as demo:
    with gr.Row():
        story = gr.TextArea(label="Duration in seconds", show_copy_button=True)
        image = gr.Image(type='pil')
        narrate = gr.Audio(label="Generated Audio", autoplay=True, streaming=True)

    async def sepia(input_img):
        chunks = []
        # Assuming chain.astream(input_img) is a method call
        async for e in chain.astream(input_img):
            chunks.append(e)
            yield "".join(chunks)

    def get2(text):
        batch_size = 3
        chunks = c.split_text(text=text)
        chunks = [chunk for chunk in chunks if chunk != "."]  # Remove periods from chunks
        for i in range(0, len(chunks), batch_size):
            batch = chunks[i:i+batch_size]
            # iterating the output from batch completion of langcahin.Runnable = runabl
            for j in runabl.batch_as_completed(batch):
                j = (j[0] + i, j[1])
                pq.put_nowait(j)
            for k in range(3):
                yield pq.get_nowait()[1]

    gr.on(fn=sepia, triggers=image.upload, inputs=image, outputs=story)

    async def get():
        global chunks, runabl
        batch_size = 3
        for i in range(0, len(chunks), batch_size):
            batch = chunks[i:i+batch_size]
            for j in runabl.batch_as_completed(batch):
                j = (j[0] + i, j[1])
                pq.put_nowait(j)
                print("hello")
            yield
        print("hello")

    async def get1():
        await get()

    def next():
        print("hello1")
        try:
            f = pq.get_nowait()[1]
        except asyncio.queues.QueueEmpty:
            return None
        return f

demo.launch()


TypeError: 'method' object is not iterable

In [7]:
text="""

In a bustling world filled with the constant hum of transportation, two trucks and two airplanes ruled the roads and skies. These mighty vehicles were the lifeblood of the economy, carrying goods and people from one place to another.

    The two trucks, named Bessie and Betty, were old friends. They had been on the road together for years, their engines purring like contented cats as they carried their precious cargo from one destination to another. Bessie was a sturdy flatbed truck, her bed filled with crates and barrels, while Betty was a sleek refrigerated truck, her cold heart keeping perishable goods fresh and delicious.

    The two airplanes, named Falcon and Falconess, were equally impressive. They soared through the skies, their wings cutting through the clouds like knives. Falcon was a cargo plane, his hold filled with pallets and containers, while Falconess was a passenger plane, her cabin filled with excited travelers, eager to explore new places.

    Every day, Bessie and Betty would make their way along the highways, their engines rumbling as they climbed hills and descended into valleys. They would pass each other on the road, exchanging friendly honks and waves, before continuing on their separate journeys.

    Meanwhile, Falcon and Falconess would take to the skies, their powerful engines roaring as they ascended into the heavens. They would fly in formation, their wings glinting in the sun as they cruised above the world, before parting ways to deliver their cargo to their respective destinations.

    And so, the two trucks and two airplanes continued their daily routine, their engines never faltering, their spirits never dimming. They were the unsung heroes of the transportation world, the unseen workers keeping the economy running smoothly. And they were proud to do it, day in and day out.

    """

In [34]:
await runabl.ainvoke(chunks[2])

(16000,
 array([ 6.4099895e-06, -3.3737240e-05, -2.8955947e-05, ...,
         4.1919535e-05,  2.9555011e-05, -1.2921644e-05], dtype=float32))

In [21]:
pq.

IndexError: list index out of range

In [128]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, TextSplitter
c = RecursiveCharacterTextSplitter(separators=["."], chunk_size=20, chunk_overlap=0)
import queue
from asyncio.queues import PriorityQueue

pq = PriorityQueue()
chunks = c.split_text(text=text)
chunks.remove(".")

# for i in range(0,len(chunks)):
#     pq.put_nowait((i,await runabl.ainvoke(chunks[i])))
# # chunks

# while not pq.empty:
#     print(pq.get_nowait(),"--")
    # if pq.empty :
    #     break
# pq.

In [114]:
import gradio as gr
import asyncio
import queue

# Create a global queue for processed items
pq = asyncio.Queue()

# Initialize global variables
chunks = []
runabl = None  # This should be assigned to your Langchain.Runnable

# Function to process the data in batches of 3
async def get():
    global chunks, runabl
    batch_size = 3
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        # Iterating over the output from batch completion of langchain.Runnable = runabl
        for j in await runabl.batch_as_completed(batch):
            # Process and append the object in order
            j = (j[0] + i, j[1])
            await pq.put(j)

# Function to handle the Gradio event
async def handle_event(duration):
    global chunks
    # Here you would create the chunks based on the duration
    # This is just a placeholder logic
    chunks = [duration] * 9  # Example: creating 9 chunks from the duration
    await get()
    return await pq.get()

# Function to handle stopping the audio
async def stop_audio(_):
    return await pq.get()

with gr.Blocks() as demo:
    with gr.Row():
        duration_input = gr.Textbox(label="Duration in seconds")
        generated_audio = gr.Audio(label="Generated Audio", autoplay=True, streaming=True)
    
    # Connecting the Gradio inputs and outputs
    duration_input.change(fn=handle_event).then(fn=lambda _: pq.get_nowait()[1], outputs=generated_audio)
    generated_audio.stop(fn=stop_audio, outputs=generated_audio)

demo.launch()



/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/utils.py:1001: UserWarning: Expected 1 arguments for function <function handle_event at 0x7f27802e1a20>, received 0.
  warnings.warn(
/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/utils.py:1005: UserWarning: Expected at least 1 arguments for function <function handle_event at 0x7f27802e1a20>, received 0.
  warnings.warn(
/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/utils.py:1001: UserWarning: Expected 1 arguments for function <function <lambda> at 0x7f27801264d0>, received 0.
  warnings.warn(
/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/utils.py:1005: UserWarning: Expected at least 1 arguments for function <function <lambda> at 0x7f27801264d0>, received 0.
  warnings.warn(
/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/utils.py:1001: UserWarning: Expected 1 arguments 

Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/helpers.py:947: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
Traceback (most recent call last):
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/queueing.py", line 541, in process_events
    response = await route_utils.call_process_api(
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/blocks.py", line 1928, in process_api
    result = await self.call_function(
  File "/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/gradio/blocks.py", line 1512, in call_function
    prediction = await fn(*processed_input)
  File "/home/kar911/Documents/DataScience_LLM/.ve

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, TextSplitter
import asyncio
from asyncio.queues import PriorityQueue

# Create the text splitter
c = RecursiveCharacterTextSplitter(separators=["."], chunk_size=20, chunk_overlap=0)

# Sample text (assuming you have a variable 'text' with the text you want to split)
# text = "Your text goes here. Make sure it is long enough to split."

# Split the text into chunks
chunks = c.split_text(text=text)

# Remove empty chunks or specific characters if needed
chunks = [chunk for chunk in chunks if chunk != "."]

# Create a PriorityQueue
pq = PriorityQueue()

async def process_chunks(chunks):
    # Process each chunk and put it into the PriorityQueue
    async for i,v in runabl.abatch_as_completed(chunks):
        await pq.put((i, v))
        print(i, v)

async def print_chunks():
    # Print chunks from the PriorityQueue
    while not pq.empty():
        i, chunk = await pq.get()
        print((i, chunk), "--")
        pq.task_done()

async def main():
    # Run both tasks simultaneously
    await asyncio.gather(process_chunks(chunks), print_chunks())

# Run the main function
if __name__ == "__main__":
    try:
        # If there is no running event loop, use asyncio.run
        asyncio.run(main())
    except RuntimeError:
        # If an event loop is already running, create tasks and run them
        loop = asyncio.get_event_loop()
        loop.create_task(main())
        loop.run_until_complete(asyncio.gather(*asyncio.all_tasks(loop)))


In [ ]:
queue.PriorityQueue

In [ ]:
while not pq.empty():
    print(pq.get())

(3, (16000, array([-1.2501415e-05, -4.4408593e-05, -3.8727343e-05, ...,
        3.9589697e-05,  2.7396680e-05, -2.0276231e-05], dtype=float32)))
(4, (16000, array([ 7.2316743e-06, -3.1632011e-05, -2.7307138e-05, ...,
        4.6607216e-05,  2.9522784e-05, -7.3716756e-06], dtype=float32)))
(5, (16000, array([-6.6566921e-05, -1.6012025e-04, -8.3505765e-05, ...,
        4.7119116e-05,  2.6652750e-05, -9.0038002e-06], dtype=float32)))
(6, (16000, array([-3.4350692e-06, -3.7854159e-05, -3.1158266e-05, ...,
        5.7494013e-05, -1.1056050e-05, -9.8604753e-05], dtype=float32)))
(7, (16000, array([-1.9910735e-06, -3.3838252e-05, -2.7976983e-05, ...,
        4.9033020e-05,  2.7951761e-05, -6.0080511e-06], dtype=float32)))
(8, (16000, array([-5.1095799e-06, -3.5330693e-05, -3.0781102e-05, ...,
        4.7544785e-05,  2.4969158e-05, -1.3209066e-05], dtype=float32)))
(9, (16000, array([-6.8212921e-06, -3.7683589e-05, -3.0825493e-05, ...,
        5.0159899e-05,  2.5722074e-05, -1.5758958e-05], dt

In [ ]:
from langchain_core.runnables import RunnableLambda

runnable = RunnableLambda(lambda x: str(x))

async def example():
    async for index, result in runnable.abatch_as_completed([1, 2, 3, 4, 5]):
        print(f"Result {index}: {result}")

await example()

Result 2: 3
Result 4: 5
Result 0: 1
Result 1: 2
Result 3: 4


In [ ]:
import queue

# Define a priority queue
pq = queue.PriorityQueue()

# Add items with a priority
pq.put((3, "Item 1"))  # (priority, data)
pq.put((1, "Item 2"))
pq.put((2, "Item 3"))

# Retrieve items in priority order
while not pq.empty():
    priority, item = pq.get()
    print(f"Priority: {priority}, Item: {item}")

Priority: 1, Item: Item 2
Priority: 2, Item: Item 3
Priority: 3, Item: Item 1


RuntimeError: asyncio.run() cannot be called from a running event loop

In [2]:
import gradio as gr
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.runnables import RunnableLambda
from langchain_huggingface import HuggingFaceEndpoint


title = "Image to short Story Generator"
description = """
Upload an image, and this app will generate a short story based on the image.
"""

Full=[]
chunks = []



pipe1 = pipeline("object-detection", model="facebook/detr-resnet-50")
pipe2 = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_length=100,
    temperature=0.5,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)
def reduce_add(a):
    ll=dict()
    for i in a:
        if i['score'] > 0.89:
            if i['label'] not in ll.keys():
                ll[i['label']] = 1
            else:
                ll[i['label']]+=1
    return "there are \n"+', \n'.join([str(i[1])+' '+i[0]  for i  in ll.items() ])

def image_segmentation_tool(image: str):
    segmentation_results = pipe1(image)
    if reduce_add(segmentation_results) == "there are \n":
        raise Passs()
    return reduce_add(segmentation_results)

def image_caption_tool(image: str):
    # image = Image.open(image_path)
    segmentation_results = pipe2(image)
    if segmentation_results[0]["generated_text"] == "":
        raise Passs("no result found use different image to create story") 
    return segmentation_results[0]["generated_text"]



def story_generation_tool(segmentation_results):
    prompt_template = """
    You are a storyteller. Based on the following segmentation results, create a story:
    {segmentation_results}

    Story:
    """
    prompt = PromptTemplate.from_template(prompt_template)
    story = prompt | llm
    return story.invoke(input={"segmentation_results":segmentation_results})

def translation_tool(english_text):
    prompt_template = """
    You are a translator. Translate the following English text to Hindi:
    {english_text}

    Translation:
    """
    prompt = PromptTemplate.from_template(prompt_template)
    translation = prompt | llm
    return translation.invoke(input={"english_text": english_text})


runnable = RunnableLambda(image_segmentation_tool).with_fallbacks([RunnableLambda(image_caption_tool)])
runnable2 = RunnableLambda(story_generation_tool)
runnable3 = RunnableLambda(translation_tool)

chain = runnable | runnable2
#-----

# async def sepia(input_img):
#     global chunks
#     v = 0
#     Full = []
#     async for e in story.astream(input={"segmentation_results": input_img}):
#         if v // 60 == 0:
#             "".join(chunks)
#             chunks.clear()
#         chunks.append(e)
#         Full.extend(chunks)
#         v += 1
#         yield "".join(Full)

# async def sepia1(input_img):
#     global chunks
#     v = 0
#     Full = []
#     async for e in story.astream(input={"segmentation_results": input_img}):
#         if v // 30 == 0:
#             "".join(chunks)
#             chunks.clear()
#         chunks.append(e)
#         Full.extend(chunks)
#         v += 1
#         yield "".join(Full)

# async def combined_sepia_functions(input_img):
#     sepia_gen = sepia(input_img)
#     sepia1_gen = sepia1(input_img)
    
#     while True:
#         try:
#             result1 = await sepia_gen.__anext__()
#         except StopAsyncIteration:
#             result1 = None

#         try:
#             result2 = await sepia1_gen.__anext__()
#         except StopAsyncIteration:
#             result2 = None

#         if result1 is None and result2 is None:
#             break

#         yield (result1, result2)

async def sepia(input_img):
    v=0
    async for e in story.astream(input={"segmentation_results":input_img}):
        if v//60== 0:
            "".join(chunks)
            chunks.clear()
        chunks.append(e)
        Full.extend(chunks)
        v=+1
        yield "".join(Full)


demo = gr.Interface(sepia, gr.Image(type='pil'), "textarea",title=title,
    description=description,live=True
)
if __name__ == "__main__":
    demo.launch()

#-----
async def generate_tone2(note):
    if note[0] > 2:
        await asyncio.sleep(5)
    d = pipe5(note[1])
    return (d["sampling_rate"], d["audio"].squeeze())

def generate_tone1(note):
    d = pipe5(note)
    return (d["sampling_rate"], d["audio"].squeeze())

async def generate_tone(note):
    yield await generate_tone2(note)

runabl = RunnableLambda(func=generate_tone1)
#-----
c = RecursiveCharacterTextSplitter(separators=["."], chunk_size=20, chunk_overlap=0)
import queue

pq = queue.PriorityQueue()
chunks = c.split_text(text=text)
chunks.remove(".")
async for i,v in runabl.abatch_as_completed(enumerate(chunks)):
    pq.put((i, v))

#-----
import gradio as gr

def generate_tone1(note):
    d = pipe5(note)
    return (d["sampling_rate"], d["audio"].squeeze())

with gr.Blocks() as demo:
    with gr.Row():
        duration_input = gr.Textbox(label="Duration in seconds")
        generated_audio = gr.Audio(label="Generated Audio", autoplay=True)
    
    duration_input.change(generate_tone1, inputs=duration_input, outputs=generated_audio)
    @gr.on(triggers=[generated_audio.stop], outputs=generated_audio)
    def greet():
        return pq.get()
    

        # yield pipe5("")

#count of RecursiveCharacterTextSplitter splits

demo.launch()

/home/kar911/Documents/DataScience_LLM/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (in

NameError: name 'HUGGINGFACEHUB_API_TOKEN' is not defined

In [ ]:
import gradio as gr
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.runnables import RunnableLambda
from langchain_huggingface import HuggingFaceEndpoint
import time
import asyncio



async def generate_tone2(note):
    if note[0] > 2:
        await asyncio.sleep(1)
    d = pipe5(note[1])
    return (d["sampling_rate"], d["audio"].squeeze())

async def generate_tone1(note):
    if note[0] > 2:
        await asyncio.sleep(1)
    d = pipe5(note[1])
    return (d["sampling_rate"], d["audio"].squeeze())

async def generate_tone(note):
    yield await generate_tone2(note)

runabl = RunnableLambda(func=generate_tone)
# runabl = RunnableLambda(func=generate_tone1)
#-----
c = RecursiveCharacterTextSplitter(separators=["."], chunk_size=20, chunk_overlap=0)
import queue

pq = queue.PriorityQueue()


#-----
import gradio as gr

def generate_tone1(note):
    d = pipe5(note)
    return (d["sampling_rate"], d["audio"].squeeze())

async def data(chunks):
    async for i,v in runabl.abatch_as_completed(enumerate(chunks)):
        pq.put((i, v))

with gr.Blocks() as demo:
    with gr.Row():
        duration_input = gr.Textbox(label="Duration in seconds")
        generated_audio = gr.Audio(label="Generated Audio", autoplay=True,streaming=True)
    
    # duration_input.change(generate_tone1, inputs=duration_input, outputs=generated_audio)
    
    @gr.on(triggers=[duration_input.change],inputs=duration_input)
    async def greet2(text):
        chunks = c.split_text(text=text)

        if "." in chunks:
            chunks.remove(".")
        data(chunks)

    # @gr.on(triggers=[duration_input.change],inputs=duration_input)
    # def greet1(text): 

    @gr.on(triggers=[generated_audio.stop], outputs=generated_audio)
    def greet():
        return pq.get()
    

        # yield pipe5("")

#count of RecursiveCharacterTextSplitter splits

demo.launch()

In [ ]:
import gradio as gr
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.runnables import RunnableLambda
from langchain_huggingface import HuggingFaceEndpoint
import time
import asyncio
import queue

# Assuming pipe5 is defined elsewhere in your code
# pipe5 = pipeline("text-to-speech")  # or whatever pipeline you are using

async def generate_tone2(note):
    if note[0] > 2:
        await asyncio.sleep(1)
    d = pipe5(note[1])
    return (d["sampling_rate"], d["audio"].squeeze())

async def generate_tone(note):
    yield await generate_tone2(note)

runabl = RunnableLambda(func=generate_tone)
c = RecursiveCharacterTextSplitter(separators=["."], chunk_size=20, chunk_overlap=0)
pq = queue.PriorityQueue()

async def data(chunks):
    async for i, v in runabl.abatch_as_completed(enumerate(chunks)):
        pq.put((i, v))

def get_audio_from_queue():
    while True:
        if not pq.empty():
            i, v = pq.get()
            yield v[1], v[0]  # Return audio data and sample rate
        else:
            time.sleep(0.1)  # Wait for more data

with gr.Blocks() as demo:
    with gr.Row():
        duration_input = gr.Textbox(label="Duration in seconds")
        generated_audio = gr.Audio(label="Generated Audio", autoplay=True, streaming=True)

    @gr.on(inputs=duration_input, triggers=duration_input.change)
    def greet2(text):
        chunks = c.split_text(text=text)
        if "." in chunks:
            chunks.remove(".")
        data(chunks)

    @gr.on(outputs=generated_audio, triggers=generated_audio.stream)
    def stream_audio():
        return get_audio_from_queue()

demo.launch()


TypeError: 'method' object is not iterable

In [ ]:
text="""

In a bustling world filled with the constant hum of transportation, two trucks and two airplanes ruled the roads and skies. These mighty vehicles were the lifeblood of the economy, carrying goods and people from one place to another.

    The two trucks, named Bessie and Betty, were old friends. They had been on the road together for years, their engines purring like contented cats as they carried their precious cargo from one destination to another. Bessie was a sturdy flatbed truck, her bed filled with crates and barrels, while Betty was a sleek refrigerated truck, her cold heart keeping perishable goods fresh and delicious.

    The two airplanes, named Falcon and Falconess, were equally impressive. They soared through the skies, their wings cutting through the clouds like knives. Falcon was a cargo plane, his hold filled with pallets and containers, while Falconess was a passenger plane, her cabin filled with excited travelers, eager to explore new places.

    Every day, Bessie and Betty would make their way along the highways, their engines rumbling as they climbed hills and descended into valleys. They would pass each other on the road, exchanging friendly honks and waves, before continuing on their separate journeys.

    Meanwhile, Falcon and Falconess would take to the skies, their powerful engines roaring as they ascended into the heavens. They would fly in formation, their wings glinting in the sun as they cruised above the world, before parting ways to deliver their cargo to their respective destinations.

    And so, the two trucks and two airplanes continued their daily routine, their engines never faltering, their spirits never dimming. They were the unsung heroes of the transportation world, the unseen workers keeping the economy running smoothly. And they were proud to do it, day in and day out.

    """